# Vectorized batch geodesics

Demonstrates the NumPy-backed helpers in `loxodrome.vectorized` for bulk operations.

Prereqs from the repo root:
- `cd experiments && uv sync --all-extras`
- launch via `uv run jupyter notebook` (or `uv run jupyter lab`).

If you're using an installed package instead of the source tree, install with `pip install "loxodrome[vectorized]"` to pull in NumPy.


In [ ]:
from __future__ import annotations

import pathlib
import sys

# Allow running the notebook from experiments/notebooks while using the local source tree.
project_root = pathlib.Path('..').resolve()
python_src = project_root.parent / 'loxodrome' / 'src'
if python_src.is_dir() and str(python_src) not in sys.path:
    sys.path.insert(0, str(python_src))

print(f'Using Python {sys.version.split()[0]}')
print(f'Notebook cwd: {pathlib.Path.cwd()}')
print(f'Import path head: {sys.path[0]}')


In [ ]:
import numpy as np

from loxodrome import Ellipsoid, Point
from loxodrome import vectorized as vz

np.set_printoptions(precision=3, suppress=True)


## Build point batches

`points_from_coords` accepts contiguous NumPy arrays or Python sequences and returns a `PointBatch` that can round-trip to NumPy or pure Python buffers.


In [ ]:
coord_pairs = np.array([
    [47.6062, -122.3321],  # Seattle
    [34.0522, -118.2437],  # Los Angeles
    [40.7128, -74.0060],   # New York City
], dtype=np.float64)

batch = vz.points_from_coords(coord_pairs)

print(f'{len(batch)} points')
print('NumPy stack:')
print(batch.to_numpy())
print('Python tuples:', batch.to_python())


## Pairwise distances and bearings

Batch operations cross the FFI boundary once and return NumPy arrays by default. `ellipsoid` is optional; WGS84 is used here.


In [ ]:
origins = vz.points_from_coords(np.array([
    [47.6062, -122.3321],  # Seattle
    [39.7392, -104.9903],  # Denver
    [25.7617, -80.1918],   # Miami
], dtype=np.float64))

destinations = vz.points_from_coords(np.array([
    [37.7749, -122.4194],  # San Francisco
    [41.8781, -87.6298],   # Chicago
    [42.3601, -71.0589],   # Boston
], dtype=np.float64))

wgs84 = Ellipsoid.wgs84()

distance_only = vz.geodesic_distance_batch(origins, destinations, ellipsoid=wgs84)
bearing_result = vz.geodesic_with_bearings_batch(origins, destinations, ellipsoid=wgs84)

print('Distances (km):', np.round(distance_only.to_numpy() / 1000, 2))
print('Initial bearings (deg):', np.round(bearing_result.initial_bearing_deg, 1))
print('Final bearings (deg):', np.round(bearing_result.final_bearing_deg, 1))


## One-to-many distances from a single origin

`geodesic_distance_to_many` keeps the origin fixed while vectorizing over the destinations.


In [ ]:
seattle = Point(47.6062, -122.3321)
west_coast = vz.points_from_coords(np.array([
    [37.7749, -122.4194],  # San Francisco
    [34.0522, -118.2437],  # Los Angeles
    [21.3069, -157.8583],  # Honolulu
], dtype=np.float64))

to_many = vz.geodesic_distance_to_many(seattle, west_coast, ellipsoid=wgs84)

for city, meters in zip(['San Francisco', 'Los Angeles', 'Honolulu'], to_many.to_python()):
    print(f'Seattle -> {city}: {meters/1000:.1f} km')


## Polygon areas in batches

Polygons mirror Arrow-style coordinate + offset buffers. Offsets are monotonically increasing with a closing value for each buffer.


In [ ]:
polygon_coords = np.array([
    [0.0, 0.0],
    [0.0, 0.02],
    [0.02, 0.02],
    [0.02, 0.0],
    [0.0, 0.0],
    [37.77, -122.52],
    [37.77, -122.48],
    [37.79, -122.48],
    [37.79, -122.52],
    [37.77, -122.52],
], dtype=np.float64)

ring_offsets = [0, 5, 10]
polygon_offsets = [0, 1, 2]

polygons = vz.polygons_from_coords(polygon_coords, ring_offsets, polygon_offsets)
areas = vz.area_batch(polygons, ellipsoid=wgs84)

print('Areas (m^2):', np.round(areas.to_numpy(), 0))
